In [4]:
## import argparse
import h5py
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from plyfile import PlyData, PlyElement

In [5]:
# parse arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('--data_dir', type=str, default='/data')
# args = parser.parse_args()
data_dir = '../data/'
data_type = 'training/reg_shot/'
data_type2 = 'test/scan_shot/'
inter = 'inter_challenge.txt'
intra = 'intra_challenge.txt'

In [6]:
def load_pairs(challenge):
    # this load pairs of string number ['000', '003']
    # challenges can be intra / inter challenge / both
    
    intraFname = ''.join([data_dir, intra])
    interFname = ''.join([data_dir, inter])
    pairs = []
    if challenge == 'intra' or challenge == 'both':
        with open(intraFname) as f:
            for line in f:
                array = line.split('_')
                array[1] = array[1][:-1]
                pairs.append(array)
    elif challenge == 'inter' or challenge == 'both':
        with open(intraFname) as f:
            for line in f:
                array = line.split('_')
                pairs.append(array)
    else:
        print('error: please select right challenge')
    return pairs

In [7]:
# def reg(p):
#     # types can be registration / all(scanned)
#     p[:] = [elem for elem in p if int(elem[0]) < 100 and int(elem[1]) < 100]
#     return p

In [8]:
def load_shot(fname):
    # f = open(fname, 'r')
    # all_cont = f.read()
    num = 0
    i = 0
    shot = []
    with open(fname) as f:
        for line in f:
            array = line.split()
#             print(len(array))
            num += len(array)
            if len(array) > 0:
                np_array = np.array(array)
                f_array = np_array.astype(float)
                shot.append(f_array)
#                 print(len(array))
#                 print("{0} - first: {1}".format(i, array[0]))
#                 print("{0} - second: {1}".format(i, array[1]))
#                 print("{0} - third: {1}".format(i, array[2]))
#                 print("{0} - forth: {1}".format(i, array[3]))
                
#                 print("{0} - last: {1}".format(i, array[355]))
#             else:
#                 print("end of file")
            i = i + 1
#     print("shot shape {0}, {1}".format(len(shot), len(shot[0])))
    shot_np = np.array(shot)
    shot_np = shot_np[shot_np[:,0].argsort()]
    shot_ret = shot_np[:, 4:]
#     print("shot shape {0}".format(shot_np.shape))
    print(shot_ret.shape)
#     print(num)
    f.close()
#     shot = np.identity(6890)
    # just returns array with (1000, array[i][0] number of descriptor length...?)
    return shot_ret

In [9]:
def compute_l2(des, des2):
    # make 1000*1000 distance matrix
    l2_dist = []
    print("des: {}".format(des.shape))
    print("des2 {}".format(des2.shape))
    for i in range(0,len(des)):
        row = []
        for j in range(0,len(des2)):
#             sub = des[i] - des[j]
# #             print(sub.shape)
#             sq_sub = np.square(sub)
# #             print(sq_sub.shape)
#             sum_sq_sub = np.sum(sq_sub)
# #             print(sum_sq_sub)
#             l2_norm = np.sqrt(sum_sq_sub)
#             print(l2_norm.shape)
            l2_norm = np.sqrt(np.sum(np.square(des[i][0:]-des2[j][0:])))
            row.append(l2_norm)
        np_row = np.array(row)
#         print(np_row.shape)
        l2_dist.append(np_row)
    np_dist = np.array(l2_dist)
    print(np_dist.shape)
    return np_dist

In [10]:
def compute_l2_pt(des_i, des2):
    lst = []
    for k in range(0, des2.shape[0]):
        l2_norm = np.sqrt(np.sum(np.square(des_i-des2[k][0:])))
        lst.append(l2_norm)
    np_lst = np.array(lst)
    return np_lst

In [11]:
def find_hitrate(matrix, k):
    # first find k nearest neighbor in index
    total_hit = 0 # total_hit = 1000
    size = matrix.shape[0]
    for i in range(size):
        # sorts index from smallest to kth smallest
        
        if k < size:
            knn = np.argpartition(matrix[i], k)[:k]
#             print(knn)
#             smallest = []
#             for j in range(len(knn)):
#                 smallest.append(matrix[i][knn[j]])
#             smallest2 = sorted(matrix[i])[:k]
#             smallest.sort()
#             print("smallest {}".format(smallest))
#             print("smallest2 {}".format(smallest2))
        # same index presents -> hit
#             print("i: {0}, knn: {1}".format(i, knn))
            if i in knn:
                total_hit += 1
        elif k == size:
            total_hit += 1
    return total_hit / size
    
    # information about argpartition
    # https://numpy.org/doc/stable/reference/generated/numpy.argpartition.html
    # https://stackoverflow.com/questions/34226400/find-the-index-of-the-k-smallest-values-of-a-numpy-array
    # if the same index(row = i) presents, then it is hit -> later will be divided by 1000

In [12]:
def find_scanned_hitrate_avg():
    # make for lb operator later
    # pair array
    c = 'both'; # inter / intra / both
    t = 'scan'; # reg (registration) / scan
    pairs = load_pairs(c) # only load when t is 'scan'
    
    # number of nearest neighbors up to k
    k = 100
        
    # load two pairs of shot descripted object
    total_hit_rate = np.zeros(100)
    for i in range(100):
        fn1 = ''.join([data_dir, data_type2, 'test_scan_d_res_', pairs[i][0], '.txt'])
        fn2 = ''.join([data_dir, data_type2, 'test_scan_d_res_', pairs[i][1], '.txt'])
        shot1 = load_shot(fn1)
        shot2 = load_shot(fn2)
        
        # get a distance map between two shots
        d_map = compute_l2(shot1, shot2)
        hit_rate = [] # this is single hit_rate map
        for j in range(k):
            hit_rate.append(find_hitrate(d_map, j))
        np_hit = np.array(hit_rate)
        print(np_hit.shape)
        # add previous hit_rate map with new hit_rate map
        total_hit_rate = np.add(total_hit_rate, np_hit)
    
    total_hit_rate = np.true_divide(total_hit_rate, 100)
    return total_hit_rate

In [10]:
# scale up by 100
# def scale_up(factor, vertArr):
#     scaleMatrix = np.identity(4)*100
#     scaleMatrix[3, 3] = 1
# #     print(scaleMatrix)
#     res = []
#     for i in range(vertArr.shape[0]):
#         v4 = vertArr[i]
#         v4 = np.append(v4, [1])
#         resCoord = np.matmul(v4, scaleMatrix)
#         res.append(resCoord[:3])
#         # convert back into nparray
#     to_ret = np.asarray(res)
#     print(to_ret)
#     return 0

In [11]:
# def dist_matrix(p1, p2):
#     # p1, p2 are 3*1 ndarray
#     v4 = np.subtract(p1, p2)
#     v4 = np.append(v4, [1])
#     M = np.matmul(v4, np.transpose(v4))

In [13]:
# import h5py file (lb)
def read_lap(i):
    try:
#         data_dir = '../data/'
        training = True
        scan = False
        k = 120
        
        data_dir2 = 'training/' if training else 'test/'
        data_dir3 = 'scan_lb/' if scan else 'reg_lb/'
        prefix = 'tr_' if training else 'test_'
        mid = 'reg_' if (training and not scan) else 'scan_'
        strk = str(k)
        num = '%03d' % i
        
        filename = "".join([data_dir, data_dir2, data_dir3, prefix, mid, strk, "_", num, ".h5"])
#         print(filename)
        
        file = h5py.File(filename)
        dkey = list(file.keys())[0]
        dset = file[dkey]
        nData = np.array(dset)
        nData = np.transpose(nData)
#         print(nData.shape)
        
        numVert = int(nData[0][0])
#         print(numVert)
        numEig = int(nData[0][1])
#         print(numEig)
        phi = nData[1]
#         print(phi)
        
        vertice = nData[2:numVert+2]
        nd_vert = np.array(vertice)
        nd_vert = nd_vert.astype(float)
        print(vertice.shape)
        
#         file = h5py.File('../data/tr_scan_010.h5', 'r')
# #         file = h5py.File(args.data_dir, 'r')
#         # assume there's only one data key
#         dkey = list(file.keys())[0]
#         print(dkey)
#         dset = file[dkey]
#         print("{0}, {1}".format(dset.dtype, dset.shape))
#         nData = np.array(dset)
#         nData = np.transpose(nData)
#         print(nData.shape)
#         numVert = int(nData[0][0])
#         numFace = int(nData[0][1])
#         vert = nData[1:numVert+1]
#         print("numVert:{0}, vert shape:{1}".format(numVert, vert.shape))
#         face = nData[numVert+1:numVert+numFace+1]
#         print("numFace:{0}, face shape:{1}".format(numFace, face.shape))
    except FileNotFoundError:
        print('Cannot find the h5py binary file.');
#     return vert, face
    return vertice

In [14]:
def vis_func_map(matrix):
#     pmatrix = matrix[0:100]
#     pmatrix = pmatrix[:, 0:100]
#     print(pmatrix)
    plt.pcolor(matrix, cmap='RdBu')
    plt.axis([0, matrix.shape[0]-1, 0, matrix.shape[1]-1])
#     norm = ply.colors.Normalize(vmin=-1, vmax=1)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()
    plt.close('all')

In [15]:
def vis_3d_shape(x, y, z, row):
    # please avoid using vis_3d_shape and vis_func_map together
    fig = plt.figure(figsize=(20, 20))
    ax = plt.axes(projection='3d')
    
    ax.scatter3D(x, y, z, c=row)
#     plt.close('all')
#     plt.colorbar()
#         ax.plot_surface(xdata, ydata, zdata, cmap=row)

In [19]:
if __name__ == "__main__":   
#     hit_rate_t = np.zeros(1000)
#     for i in range(len(pairs)):
#         # load two elements
#         filename = ''.join([data_dir, data_type, 'tr_reg_res_', pairs[i][0], '.txt'])
#         filename2 = ''.join([data_dir, data_type, 'tr_reg_res_', pairs[i][1], '.txt'])
#         p1 = load_shot(filename)
#         p2 = load_shot(filename2)
#         n1 = np.array(p1)
#         n2 = np.array(p2)
#         n1 = n1.astype(float)
#         n2 = n2.astype(float)
#         dist_mat = compute_l2(n1, n2)
# #         print(dist_mat.shape)
        
#         hit_rate = []
#         for j in range(0, dist_mat.shape[0]):
#             hit_rate.append(find_hitrate(dist_mat, j))
#         np_hit = np.array(hit_rate)
#         hit_rate_t = hit_rate_t + np_hit
#     hit_rate_t = hit_rate_t * (1/dist_mat.shape[0])
    
#     plt.plot(hit_rate_t)
#     plt.show()
#     print(hit_rate_t)
    
    action1 = read_lap(0)
    print(action1.shape)
    action_t1 = torch.from_numpy(action1)
    print(action_t1.size())
    
    action2 = read_lap(3)
    actstrn1 = '%03d' % 0
    actstrn2 = '%03d' % 3
    
    ##
    # reading tr_reg_001.ply
#     fnply1 = ''.join([data_dir, 'training/reg_100/', 'tr_reg_', actstrn2, '.ply'])
#     pd = PlyData.read(fnply1)
# #     print(plydata)
#     print(pd.elements)
#     x_coord = pd['vertex'].data['x']
#     y_coord = pd['vertex'].data['y']
#     z_coord = pd['vertex'].data['z']
    
    
#     actfn1 = ''.join([data_dir, data_type, 'tr_reg_res_', actstrn1, '.txt'])
#     actfn2 = ''.join([data_dir, data_type, 'tr_reg_res_', actstrn2, '.txt'])
#     actshot1 = load_shot(actfn1)
#     actshot2 = load_shot(actfn2)
    
#     shot_cmd = find_scanned_hitrate_avg()
#     plt.plot(shot_cmd)
#     plt.show()
    
    ## to view point distance from one point in other shape
#     dist_point = compute_l2_pt(actshot1[0], actshot2)
#     vis_3d_shape(x_coord, y_coord, z_coord, dist_point)
    
    ## to view functional map
#     # action1, action2
#     ident = np.identity(6890)
#     proj1 = np.dot(np.transpose(actshot1), action1)
#     print(proj1.shape)
#     proj2 = np.dot(np.transpose(actshot2), action2)
    

#     res = np.linalg.lstsq(proj1, proj2, rcond=1)
#     ct = res[0]
#     c = np.transpose(ct)
#     vis_func_map(c)
    
#     # scale by 100 should be done before shot/lb calculation
#     dist_matrix = compute_l2(actshot1[0:100], actshot2[0:100])
# #     print(dist_matrix)
#     # right now, getting the hit rate for one object, but will average them later
    
#     hr = find_hitrate(dist_matrix, 3)
#     print(dist_matrix.shape)
#     hrs = []
#     for i in range(dist_matrix.shape[0]):
#         hr = find_hitrate(dist_matrix, i)
#         hrs.append(hr)
#     plt.plot(hrs)
#     plt.show()
#     print(hrs)

(6890, 120)
(6890, 120)
torch.Size([6890, 120])
(6890, 120)


<ipython-input-13-447d96a5a12d>:19: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  file = h5py.File(filename)


### 